In [ ]:
import pandas as pd
import numpy as np

 !pip install yfinance
import yfinance as yf


### **Mathematical Background:**

The covariance of an asset with a portfolio is the weighted average of asset's covariance to all the asset making up the portfolio.

$Cov(R_1,R_P) = Cov(R_1, w_1R_1 + w_2R_2) = w_1Cov(R_1,R_1) + w_2Cov(R_1,R_2) $

Portfolio variance is given by:
$ Var(R_p) = \sigma^2 = w'\Omega w $ ,  
where $\Omega$ is the covariance matrix of the return.
The **volatility** is given by the square root of the variance.

## **Mean-Variance model**

Markowitz's theory make 3 key assumptions:


*   Allocation is done by rational investors ( for the same return , they will choose the one with the lowest variance)
*   Capital markets are perfect
*   Returns are **normally distributed**"

Idealy, investors seek to reduce variance by diversifying their portfolio investments. Here, I will propose an arbitrary portfolio, made of SP500 companies, which is diversified across sectors (industries).

According to Markowitz, the level of investment in a particular financial asset should be based upon that assets' contribution to the distribution of the overall portfolio (covariability of assets' returns.

The efficient frontier represents the optimal retuns for each level of risk ($\sigma$).
Along the efficient frontier, the only way to achieve higher expected returns is by increasing the riskiness of the portfolio ($\sigma$).

A critical input to the mean-variance model is the estimated correlation between assets (even more if using ETF).



## **Capital Asset Pricing Model: (CAPM)**



CAPM shows the relation between the risk and expected return of a risky asset. The risky asset can be decomposed into 2 components: 


1.   Systematic risk = market risk (non-diversifiable)
2.   Specific risk = asset risk (diversifiable)

The CAPM demonstrates that by combining assets into a portfolio, each assets' unique risk can be eliminated. This leaves market risk as the portfolio's sole exposure.

CAPM requires solid asumptions:


*   Access to information for all participant equally
*   All participant make investment based on the mean and variance of returns (particulary false for retail traders by the way)
*   No frictions (taxes, cost etc ..) -> I wish
*   All participant can lend and borrow at a common risk free rate ( 3-month US treasury bond rate)
* Any individual investor's allocation cannot change the market prices

In this model, each investor's portfolio is  combination of assests specific risk and systematic risk.

Given an asset, its systematic risk is given by $\beta_i$ and is calculated as:

$\beta_i = \frac{cov(R_i, R_m)}{\sigma_m^2}$

It is very common to take the variance of the index for the market variance (ie: $\sigma_{SP500} = \sigma_m$) although subject to debate in the litterature.





In [ ]:
Tickers = ['NOW', 'AMD', 'FOX', 'JPM', 'MSCI', 'GE', 'RMD', 'BIO', 'APD', 'REG', 'MPC', 'BKNG'] #my portfolio tickers (arbitrary in SP500 list)
historical_data = yf.download(tickers= Tickers, period = "3y",group_by='ticker')